<a href="https://colab.research.google.com/github/Pratiksha100/AnalyticsProject/blob/akshay%2F443308/Concord_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/jvkersch/pyconcorde

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-req-build-4y2wcpua
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-req-build-4y2wcpua
  Resolved https://github.com/jvkersch/pyconcorde to commit 71232a8043a3ca79a7a6da2f48e9d89d81d37163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-cp310-cp310-linux_x86_64.whl size=2494564 sha256=5a4a3797330a106822deef1d567e996452c702e5f9fb7b313704d4160d998bec
  Stored in directory: /tmp/pip-ephem-wheel-cache-_0x9uhk4/wheels/be/9f/eb/e4f95e06e62f057a045679e4940e536d1b251d1ab4859c6dcc
Successfully built pyconcorde


In [ ]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Testing the concord solver on a sample file
from concorde.tsp import TSPSolver
from concorde.tests.data_utils import get_dataset_path

fname = '/content/drive/MyDrive/analytics/si1000.tsp'
solver = TSPSolver.from_tspfile(fname)
solution = solver.solve()

print(solution.found_tour)

print(solution.optimal_value)

print(solution.tour)

True
92650.0
[0 1 2 ... 5 4 3]


In [ ]:
import numpy as np
def create_dist_mat_from_atsp(path_of_file): #input : path of the .atsp file
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat #returns the distance matrix

In [ ]:


def create_dist_mat_from_atsp(path_of_file): #input : path of the .atsp file
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    # return list_of_lists    

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list
    # return lst

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    #print the length of the list
    print(n)
    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat #returns the distance matrix as a numpy array

def make_symmetric(distances): #input a assymentric distance matrix as a numpy array
    n = len(distances)
    distances_converted = []
    bigM = 1000000

    for cnt1 in range(0, n):
        tmp = []
        for cnt2 in range(0, n):
            tmp.append(bigM)

        for cnt2 in range(0, n):
            if cnt1 == cnt2:
                tmp.append(-bigM)    # from node to dummy node
            else:
                tmp.append(distances[cnt2][cnt1])

        distances_converted.append(tmp)

    # lower part of matrix (outgoing edges)
    for cnt1 in range(0, n):
        tmp = []
        for cnt2 in range(0, n):
            if cnt1 == cnt2:
                tmp.append(-bigM)    # from node to dummy node
            else:
                tmp.append(distances[cnt1][cnt2])

        for cnt2 in range(0, n):
            tmp.append(bigM)
        distances_converted.append(tmp)

    # check that matrix is symmetric
    for cnt1 in range(0, 2 * n):
        for cnt2 in range(0, 2 * n):
            assert (distances_converted[cnt1][cnt2] == distances_converted[cnt2][cnt1])

    return distances_converted #return a symmentric distance matrix



In [ ]:


def generate_tsp_file(dist_mat, filename, num_cities):    
    # Write the coordinates to a .tsp file
    fname = filename.split("/")[-1].split(".")[0]
    with open(filename, 'w') as f:
        f.write(f"NAME: {fname}\n")
        # f.write("COMMENT: Randomly generated TSP instance\n")
        f.write("TYPE: TSP (M.~Hofmeister)\n")
        f.write(f"DIMENSION: {num_cities*2}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
        f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
        f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
        # f.write("NODE_COORD_SECTION\n")
        f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
        f.write("EDGE_WEIGHT_SECTION\n")
        for i in range(len(dist_mat)):
          for j in range(len(dist_mat)):
            f.write(f'{str(dist_mat[i][j])}\t')
          f.write("\n")
        f.write("EOF\n")

In [ ]:
distances = create_dist_mat_from_atsp("/content/drive/MyDrive/analytics/br17.atsp")

new_dist = make_symmetric(distances)

generate_tsp_file(new_dist,"/content/drive/MyDrive/analytics/br17.tsp",17)


/content/drive/MyDrive/analytics/br17.atsp
4624
No of cities :  34
(68, 68)


In [ ]:
from concorde.tsp import TSPSolver
from concorde.tests.data_utils import get_dataset_path



fname = '/content/drive/MyDrive/analytics/br17.tsp'
solver = TSPSolver.from_tspfile(fname)
solution = solver.solve()

print(solution.found_tour)

print(solution.optimal_value +17*1000000)

print(solution.tour)

True
31000032.0
[ 0 17  3 28 27 32 31 33 14 24 23 25 26 18  7 29 22 30 11 19 21 15 20 10
  8  5  1  4 13 16 12  9  6  2]


In [ ]:
#automation code for checking all the instances
import os
import glob
import re
import time

base_path = os.chdir('/content/drive/MyDrive/analytics')#use os.getcwd() in windows
atsp_files =  glob.glob("/content/drive/MyDrive/analytics/*.atsp", recursive=True)
print(atsp_files)
for f in atsp_files:
  filename = f.split('/')[-1]
  print(filename)
  nu = re.findall(r'\d+', filename)
  cities = [int(num) for num in nu][0]
  
  print(cities)
  
  distances = create_dist_mat_from_atsp(f)

  new_dist = make_symmetric(distances)

  generate_tsp_file(new_dist,f,cities)

  start = time.time()  
  solver = TSPSolver.from_tspfile(f)
  solution = solver.solve()
  end = time.time()

  print(solution.found_tour)

  print(solution.optimal_value +cities*1000000)

  print(solution.tour)

  print("Total time : " , end - start , "secs\n\n")



['/content/drive/MyDrive/analytics/ft53.atsp', '/content/drive/MyDrive/analytics/ft70.atsp', '/content/drive/MyDrive/analytics/p43.atsp', '/content/drive/MyDrive/analytics/ry48p.atsp', '/content/drive/MyDrive/analytics/rbg323.atsp', '/content/drive/MyDrive/analytics/rbg358.atsp', '/content/drive/MyDrive/analytics/rbg403.atsp', '/content/drive/MyDrive/analytics/rbg443.atsp', '/content/drive/MyDrive/analytics/ftv171.atsp', '/content/drive/MyDrive/analytics/ftv34.atsp', '/content/drive/MyDrive/analytics/ftv36.atsp', '/content/drive/MyDrive/analytics/ftv39.atsp', '/content/drive/MyDrive/analytics/ftv45.atsp', '/content/drive/MyDrive/analytics/ftv48.atsp', '/content/drive/MyDrive/analytics/ftv56.atsp', '/content/drive/MyDrive/analytics/ftv65.atsp', '/content/drive/MyDrive/analytics/ftv71.atsp', '/content/drive/MyDrive/analytics/kro100p.atsp', '/content/drive/MyDrive/analytics/br17.atsp']
ft53.atsp
53
/content/drive/MyDrive/analytics/ft53.atsp
2809
No of cities :  53
(53, 53)
